In [ ]:
import numpy as np
import pandas as pd
import difflib
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity

In [ ]:
movies_data = pd.read_csv('/content/movies.csv')

In [ ]:
movies_data.head()


In [ ]:
movies_data.shape

(4803, 24)

In [ ]:
selected_features =  ['genres','keywords','tagline','cast','director']
for feature in selected_features:
  movies_data[feature] = movies_data[feature].fillna('')

In [ ]:
combined_features = movies_data['genres']+' '+movies_data['keywords']+' '+movies_data['tagline']+' '+movies_data['cast']+' '+movies_data['director']
print(combined_features)


0       Action Adventure Fantasy Science Fiction cultu...
1       Adventure Fantasy Action ocean drug abuse exot...
2       Action Adventure Crime spy based on novel secr...
3       Action Crime Drama Thriller dc comics crime fi...
4       Action Adventure Science Fiction based on nove...
                              ...                        
4798    Action Crime Thriller united states\u2013mexic...
4799    Comedy Romance  A newlywed couple's honeymoon ...
4800    Comedy Drama Romance TV Movie date love at fir...
4801      A New Yorker in Shanghai Daniel Henney Eliza...
4802    Documentary obsession camcorder crush dream gi...
Length: 4803, dtype: object


In [ ]:
vectorizer = TfidfVectorizer()
feature_vectors = vectorizer.fit_transform(combined_features)

In [ ]:
similarity=cosine_similarity(feature_vectors)

In [ ]:
import joblib

In [ ]:
joblib.dump(similarity,'similarity.pkl')
joblib.dump(movies_data,'movies_data.pkl')

['movies_data.pkl']

In [ ]:
import gradio as gr
movies_data = joblib.load('movies_data.pkl')
similarity= joblib.load('similarity.pkl')

def recommend_movies(movie_name,n):
  list_of_all_titles = movies_data['title'].tolist()
  find_close_match = difflib.get_close_matches(movie_name, list_of_all_titles)
  close_match = find_close_match[0]
  index_of_the_movie = movies_data[movies_data.title == close_match]['index'].values[0]
  similarity_score=list(enumerate(similarity[index_of_the_movie]))
  sorted_similar_movies = sorted(similarity_score, key = lambda x:x[1], reverse = True)
  recommendations=[]
  i=1
  for movie in sorted_similar_movies:
    index= movie[0]
    title=movies_data[movies_data.index== index]['title'].values[0]
    if i <= n and title!= close_match:
      recommendations.append(f"{i}.{title}")
      i +=1
  return "\n".join(recommendations)
iface = gr.Interface(
    fn=recommend_movies,
    inputs=[
        gr.Textbox(label="Enter the movie name"),
        gr.Radio(choices=[5,10,20,50],label="Number of recommendations",value=10)
        ],
    outputs=gr.Textbox(label="Recommended Movie"),
    title="MovieRecommender")
iface.launch(share=True,debug=True)

Colab notebook detected. This cell will run indefinitely so that you can see errors and logs. To turn off, set debug=False in launch().
* Running on public URL: https://ac8b6e47e922ac2c97.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)
